# 29. 활성화 함수의 이해

## 1. 들어가며

### 목차
---
* 활성화 함수
    - 활성화 함수
    - 퍼셉트론
        - 신경세포의 구조
        - 퍼셉트론의 구조
* 선형과 비선형
    - 선형
    - 비선형
    - 비선형 함수를 쓰는 이유
        - 그렇다면..(비선형 함수를 쓴다면?)
* 활성화 함수의 종류
    - 이진 계단 함수
        - 이진 계단 함수의 한계
    - 선형 활성화 함수
        - 선형 활성화 함수의 한계
    - 비선형 활성화 함수
        - 시그모이드/로지스틱
        - 하이퍼볼릭 탄젠트
        - ReLU
        - ReLU의 단점을 극복하기 위한 시도들
* 끝으로…

### 준비물
---
* 사용할 모듈 미리 import하기!

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from itertools import product
import tensorflow as tf

tf.random.set_seed(7879)

실습에 사용할 사진을 미리 다운받아서 작업 디렉토리에 저장하자.

```bash
$ wget https://aiffelstaticprd.blob.core.windows.net/media/original_images/jindo_dog.jpg
$ mkdir -p ~/AIFFEL/Fundamentals/F29_Activate_functions/activation
$ mv jindo_dog.jpg ~/AIFFEL/Fundamentals/F29_Activate_functions/activation
```

## 2. 활성화 함수
오늘은 수학 분야에서도 딥러닝과 아주아주 밀접하고 직접적인 주제를 다루어보겠다. 바로 softmax나 ReLU 등 이미 익숙히 들어보았을 __활성화 함수(activation function)__이다.

우리가 지금까지 써왔던 신경망 속의 퍼셉트론(perceptron) 혹은 노드(node)도 '특정 조건'이 만족하면 '활성화' 되도록 디자인되어 있다. 노드에 입력으로 들어오는 값이 어떤 '임계치'를 넘어가면 "활성화(activated)"되고, 넘어가지 않으면 "비활성화(deactivated)"되게끔 코딩이 되어있는 것이다. 익숙하게 써왔던 ReLU 함수는 이러한 관점에서 어떻게 작용하고 있을까?

ReLU는 입력값이 음수라면, 즉 0 미만이라는 조건을 만족한다면 0을 출력하고, 입력값이 0이상이면 입력값 그대로를 출력하도록 디자인되어있다. 이를 "활성화 or 비활성화"로 표현하려면 우선 활성화되는 기준을 정해야한다. 만일 출력값이 0보다 큰 경우를 활성화 되었다고 정의한다면, ReLU함수란, "0 미만인 경우는 비활성화가 되고 0 이상인 경우는 활성화되는 함수"라고 말할 수 있다.

시그모이드는 ReLU처럼 명확하고 간단하게 글로 표현하긴 힘들지만, 입력값이 $- \infty$로 갈수록 0을 출력하고 $+ \infty$로 갈수록 1을 출력하며, 0일 때는$\frac{1}{2}$을 출력하는 함수이다. 수식적으로는 $x$를 입력값이라고 할 때, $f(x)=\frac{1}{1+exp(-x)}$로 표현할 수 있다.<br>
이 함수는 입력값에 '무관하게' $0\sim 1$사이의 값으로 출력하는 특징이 있으며, 따라서 "참" / "거짓" 혹은 "앞면" / "뒷면" 처럼 2가지 상황을 구분할 때 용이하다. (참=0, 거짓=1로 대응 시켜 구분할 수 있으므로) 

또한 활성화와 비활성화의 조건을 0.5로 잡았으니, 시그모이드는 0이상일 때 활성화가 되고, 0보다 작을 때 비활성화된다고 말할 수 있다.

<img src="./image/sigmoid.png" alt="sigmoid" />

활성화 함수의 기본적 정의는 위와 같지만, 실제로 딥러닝에서 활성화 함수를 쓰는 결정적 이유는 따로 있다. 바로 딥러닝 모델의 표현력을 향상 시켜주기 위해서 이다. 전문적인 용어로는 모델의 __representation capacity__ 또는 __expressivity__ 를 향상 시킨다라고도 말힌다.

활성화 함수는 모델의 표현력을 왜 향상시켜줄까? 답은 간단하다. 만일 어떤 모델이 $w_{1}$, $b_{1}$이라는 2개의 parameter로 이루어진 다음과 같은 모델이라고 해보겠다.

$$ f(x)=xw_{1}+b_{1}$$

그런데 이 모델로 $x^{2}$, $x^{5}$, $sin(x)$등으로 표현되는 데이터를 햑습할 수 있을까? 답은 "그럴 수 없다" 이다. 왜냐하면, $w_{1}$, $b_{1}$값을 아무리 바꿔도$x^{2}$, $x^{5}$, $sin(x)$와 같은 함수는 절대 표현할 수 없기 때문이다. 이를 수학적으로 말하면, '"선형" 함수(직선)로는 "비선형"함수(사인곡선 or $x^{5}$와 같은 고차항)을 표현할 수 없다'라고 말한다.

그런데 잘 생각해보면 딥러닝 모델의 parameter($x$,$b$)들은 입력값 $x$와 선형 관계이다. 왜냐하면, $xw+b$의 표현되는, 즉 곱하고 더하는 연산만 하면서 그다음 layer로 전달하기 때문이다. 그리고 아무리 많은 layer들을 겹쳐도 역시 그 결과는 선형관계이다. 따라서 사인 곡선처럼 직선으로는 근사시킬 수 없는 (혹은 고양이나 강아지 사진처럼 무수히 많고 복잡한 특징들을 가진) 비선형 데이터를 표현하려면 딥러닝 모델도 비선형성을 지니고 있어야 한다. 이때 쓰인 것이 바로 활성화 함수이고, 이 활성화 함수를 layer 사이 사이에 넣어줌으로써 모델이 비선형 데이터도 표현할 수 있게 되었다. 

## 3. 퍼셉트론
활성화 함수라는 아이디어는 어디서부터 비롯되었는지 알아보자.

### 퍼셉트론
---
활성화 함수가 어디서 나온 건지 알아보기 위해 먼저 퍼셉트론에 대해 짧게 공부하고 가겠다.

우리가 알고 있는 딥러닝 모델은 보통 여러 개의 층으로 이루어져 있다. 그중에 하나의 층을 가져와 다시 쪼갠다면 보통 '노드'라고 불리는 것으로 쪼개지게 되는데, 이것이 바로 __퍼셉트론(Perceptron)__이다. 반대로 퍼셉트론을 쌓으면 단층 퍼셉트론이 되고, 이 단층 퍼셉트론을 쌓으면 다층 퍼셉트론이 된다. 즉, 우리가 알고 있는 딥러닝 모델은 사실 퍼셉트론들이 다양한 구조로 쌓인 것이다.

그렇다면 퍼셉트론은 어떻게 나온 것일까?

우리의 목표는 컴퓨터를 학습시키는 것이다. 그런데 어떻게 컴퓨터를 학습시킬 수 있을까? 과학자들은 방법을 생각하다가 이미 __잘 짜여진 머신__이라고도 할 수 있는 동물의 학습 방법을 모방하기로 한다. 이를 위해서 학습시킬 머신 자체를 동물의 신경세포와 유사하게 설계해 나오게 된 것이 바로 최초의 퍼셉트론이다.

* 참조 : [History of the Perceptron](https://web.csulb.edu/~cwallis/artificialn/History.htm)

### 신경세포의 구조
---
그럼 퍼셉트론의 구조를 보기 전에 간단히 __신경세포(Nueron)__에 대해 잠시 살펴보겠다.

신경세포는 크게 세포체(Soma), 가지돌기(Dendrite), 축삭돌기(Axon), 시냅스(Synapse)로 구성되어 있다.<br>
_(물론 해부학적으로 더 세분화해서 나눌 수 있겠지만 여기서는 퍼셉트론과 대응되는 구성요소만 다루겠다.)_

<img src="./image/neuron.png" alt="Nueron" />

* __가지돌기__
세포로 전달되는 신호를 받아들이는 부분이다.
* __축삭돌기__
세포에서 다른 세포로 신호를 전달하는 부분이다.
* __시냅스__
가지돌기와 축삭돌기 사이에 있는 부분으로 신호 전달의 세기를 담당하며 시냅스의 발달 정도에 따라 같은 신호도 강하게 전달되거나 약하게 전달된다. 시냅스는 사용 빈도에 따라 커지거나 작아지며 심지어 시냅스 자체가 사라지기도 한다.
* __세포체__
세포체는 각 가지돌기로부터 들어온 신호들로부터 자신의 출력 신호를 만들고 이를 다음 세포에 전송한다. 이때의 출력 신호는 단순히 입력신호들의 합이 아닌 비선형 방식으로 결정된다. 즉, 합이 일정 수준 이하이면 비활성 상태가 되어 신호를 무시하고, 일정 수준을 넘게 되면 활성 상태가 되어 신호를 다음 세포로 전송한다.

### 퍼셉트론의 구조
---
<img src="./image/perceptron.png" alt="Perceptron" />

앞에서 소개한 신경세포의 구조와 퍼셉트론의 구조를 대응 시켜 소개하겠다.

퍼셉트론에 주어지는 입력은 신경세포에서 가지돌기로 받아들이는 신호에 해당한다. 각 입력 항에 곱해지는 가중치는 신경 세포들의 연결 부위에 형성된 시냅스의 발달 정도에 해당한다. $x_{i}w_{i}$값들은 각 가지돌기에서 시냅스를 거쳐서 오는 신호들에 해당되며 세포체에서 이 신호들은 합쳐($\Sigma$)진다. 실제 신경세포에는 없지만 퍼셉트론에선 추가로 편향이라고 불리는 $b$라는 신호 또한 합산에 포함된다. 그리고 이 합쳐진 신호는 세포체에서 신호를 처리하는 방식과 비슷하게 적절한 __활성화 함수(activation function)__ 를 거쳐 출력이 결정된다. 이것을 식으로 정리하면 다음과 같다.

$$X=\sum_{i=1}^{n}x_{i}w_{i} + bY=f(X)$$

여기서 $X$는 합쳐진 신호이며, $Y$는 퍼셉트론의 최종 출력이다. 당연한 말이지만, 이런 간단한 모델이 복잡한 신경 세포와 동일하게 작동된다고 말할 수 없다. 하지만 이 퍼셉트론을 다양한 구조로 연결하고, 가중치와 편향값을 적절히 조정해주는 것으로 유사하게 작동하게 만들어 줄 수 있다. 이렇게 조정해주는 과정이 학습이며 학습을 통해 기계는 동물과 비슷한 일들(과일 분류, 결함 탐지, 음성 인식 등)을 처리할 수 있게 된다.

_(요즘 사용되는 퍼셉트론의 구조는 처음 발표된 것과 좀 달라졌지만 이름은 '퍼셉트론' 그대로 사용하고 있다.)_

<img src="./image/compare.png" alt="compare" />

### 활성화 함수
---
앞서 확인할 수 있듯이 활성화 함수는 신경 세포로 치면 __세포체__에서 일어나는 일을 맡고 있다. 따라서 하는 일도 비슷하다. __들어온 신호가 특정 임계점을 넘으면 출력을 하고, 넘지 못하면 무시__를 한다. 활성화 함수는 신호를 '전달' 해주기 때문에 __Transfer function__으로도 알려져있다.

활성화 함수는 그 표현에 따라

* __선형 활성화 함수(Linear activation function)__
* __비선형 활성화 함수(Non-linear activation function)__

로 나눌 수 있다.

딥러닝에서는 일반적으로 비선형 활성화 함수를 사용한다. 그럼 선형 활성화 함수는 왜 딥러닝에서 사용되지 않는 걸까? 이를 알아보기 위해 먼저 __선형(Linear)__에 대해 알아보겠다.

## 4. 선형과 비선형
__선형__과 __비선형__, 각각은 무엇을 뜻하는 걸까?

### 선형(Linear)
---
먼저 참고 영상을 보자.

[![Linear transformations](http://img.youtube.com/vi/kYB8IZa5AuE/0.jpg)](https://youtu.be/kYB8IZa5AuE) 

영상에서 알 수 있듯이 선형 변환이란 '선형'이라는 규칙을 지키며 $V$ 공간상의 벡터를 $W$ 공간상의 벡터로 바꿔주는 역할을 한다.

<img src="./image/transformation.png" alt="transformations" />

그럼 자세하게 들어가서 먼저 선형 변환(linear transformation)이 어떤 것인지 __정의__하고 가겠다.

#### [선형 변환 정의]

$V$와 $W$가 어떤 $^{(1)}$벡터 공간이고 둘 모두 $^{(2)}$실수 집합$^{(3)}$상에 있다고 가정하겠다. 이 때 함수 $^{(4)}T:V\rightarrow W$가 다음 두 조건을 만족할 때,

* 가산성(Additivity) : 모든$x,y \in V$에 대해, $\tau (x+y)=\tau (x)+ \tau (y)$
* 동차성(Homogeneity) : 모든 $x \in V$, $c \in \mathbb{R}$ 에 대해, $ \tau (cx)=c \tau (x)$

우리는 함수 $ \tau $를 __선형 변환(linear transformation)__이라고 부른다.<br>
$^{(1)}$: 간단하게 말해서 벡터를 그릴 수 있는 공간입니다. 영상에서의 좌표 평면이라고 생각하면 된다.<br>
$^{(2)}$: 정확히 표현하면 같은 __체(field)에 속해 있다__고 해야 하나, 이 글에선 실수만 다루기 때문에 실수 집합 상에 있다고 표현했다. 체의 예로는 실수 집합 $\mathbb{R}$, 유리수 집합 $\mathbb{Q}$, 복소수 집합 $\mathbb{C}$ 등이 있다.<br>
$^{(3)}$: **실수 집합 상에 있다**는 말은 $V$를 이루는 **원소들이 실수**라는 의미이다. 예를 들어 실수 집합 상의 $V$가 어떤 벡터들의 집합이라고 했을 때, 그 백터는 실수 벡터(벡터의 각 원소가 실수)가 된다.<br>
$^{(4)}$: 정의역(domain)이 $V$이고 공역(codomain)이 $W$인 함수 $ \tau $라는 의미이다.

<img src="./image/linear.png" alt="linear" />

간단히 '$\tau$는 __선형(linear)__이다.' 라고 하기도 한다.

$\tau$가 선형이라면 다음과 같은 성질을 가진다.

* $\tau(0)=0$
* 모든 $x,y \in V$와 $c \in \mathbb{R}$에 대해 다음 식과 동치이다.

$$ \tau (cx+y)=c \tau (x)+ \tau (y)$$

* 모든 $x,y \in V$에 대해 $ \tau (x,y)= \tau (x)- \tau (y)$
* $x_{1}, x_{2},..., x_{n} \in V$과 $a_{1}, a_{2},..., a_{n} \in \mathbb{R}$에 대해 다음의 식과 동치이다.

$$ \tau (\sum_{i=1}^{n}a_{i}x_{i}) = \sum_{i=1}^{n}a_{i} \tau (x_{i})$$

일반적으로 주어진 함수가 선형인지 확인해 볼 때는 2번째 성질을 이용한다.

$$ \tau (cx+y)=c \tau (x)+ \tau (y)$$

예를 하나 들어보겠다.

다음과 같이 정의된 함수$ \tau :R^{2} \rightarrow R^{2}$는 선형일까?

$$  \tau (a_{1}, a_{2}) = (a_{1} + 2a_{2}, a_{2})$$

$c \in \mathbb{R}$이고 $(x_{1}, x_{2}), (y_{1}, y_{2}) \in \mathbb{R}^{2}$라고 하겠다. 그럼,

$$c(x_{1}, x_{2}) + (y_{1}, y_{2}) = (cx_{1} + y_{1}, cx_{2} + y_{2})$$

이므로, 이를 이용해서 $ \tau (c(x_{1}, x_{2}) + (y_{1}, y_{2}))$를 구하면 다음과 같다.

$$ \tau (c(x_{1}, x_{2}) + (y_{1}, y_{2})) =  \tau (cx_{1} + y_{1}, cx_{2} + y_{2}) \\ = (cx_{1} + y_{1} + 2(cx_{2} + y_{2}), cx_{2} + y_{2})$$

또한,

$$c \tau (x_{1}, x_{2}) +  \tau (y_{1}, y_{2}) = c(x_{1} + 2x_{2}, x_{2}) + (y_{1} + 2y_{2}, y_{2}) \\ = (cx_{1} + 2cx_{2} + y_{1} + 2y_{2}, cx_{2} + y_{2}) \\ = (cx_{1} + y_{1} + 2(cx_{2} + y_{2}), cx_{2} + y_{2})$$

이므로, $ \tau (c(x_{1}, x_{2}) + (y_{1}, y_{2})) = c \tau (x_{1}, x_{2}) +  \tau (y_{1}, y_{2})$이다. 따라서 2번째 성질에 의해 $ \tau $는 __선형__이다.

## 5. 비선형 함수를 쓰는 이유 (1)

### 비선형(Non-linear)
---
그렇다면 비선형은 뭘까?

간단하다. 선형이 아닌 함수를 __비선형(Non-linear)__ 함수라고 한다.

아래 함수 $f(x)$들을 살펴보고 선형인지 비선형인지 알아보자.

1. $f(x) = 3x$
2. $f(x) = x^{2}$
3. $f(x) = \theta_{0}x_{0} + \theta_{1}x_{1},(x = [x_{0}\,  x_{1}]은\, 벡터)$

* 1) 어떤 실수 $x, y, c$가 있다고 할 때, $f(cx + y) = 3(cx + y)$이고, $cf(x) + f(y) = 3cx + 3y = 3(cx + y)$이므로 $f$는 선형이다.
* 2) 어떤 실수 $x, y, c$가 있다고 할 때, $f(cx + y) = (cx + y)^{2}$이고, $cf(x) + f(y) = cx^{2} + y^{2}$이므로 $f$는 선형이 아니다.
* 3) 주어진 식을 벡터의 형태로 다음과 같이 표현할 수 있다.

$$f(x) = \theta_{0}x_{0} + \theta_{1}x_{1} = [\theta_{0}\, \theta_{1}] \cdot [x_{0}\,  x_{1}] = \theta x$$

어떤 벡터 $x, y \in \mathbb{R}^{2}$와 어떤 실수 $c$가 있다고 할 때,

$$f(cx + y) = f([cx_{0} + y_{0} \, \, cx_{1} + y_{1}]) \\ = [\theta_{0}\, \theta_{1}] \cdot [cx_{0} + y_{0} \, \, cx_{1} + y_{1}] \\ = \theta_{0}(cx_{0} + y_{0}) + \theta_{1}(cx_{1} + y_{1}) $$

이므로 $f$는 선형이다.

### 비선형 함수를 쓰는 이유
---
그렇다면 왜 딥러닝에서는 비선형 활성화 함수를 주로 사용할까?

앞서 Introduction에서 잠시 언급했듯, 한 문장으로 요약하자면, "딥러닝 __모델의 표현력을 향상__시키기 위해서" 이다.

* 그럼 선형 활성화 함수를 사용하면 왜 표현력이 떨어지게 되는 걸까?
* 레이어를 충분히 쌓는다면 선형 활성화 함수를 사용한 모델의 표현력을 향상시킬 수 있지 않을까?

간단한 예시를 통해 알아가보도록 하겠습니다.

<img src="./image/function.png" alt="function" />

위 그림과 같이 퍼셉트론 3개로 구성된 모델이 있다고 가정하겠습니다.

입력값 $x$가 모델을 거치면 출력값 $y$가 된다. (여기서 입력값 $x$와 출력값 $y$는 스칼라값이고 $f$는 활성화 함수이다.)

수식으로 표현하면 다음과 같다.

$$y = f(w_{3}f(w_{2}f(w_{1}x))) = f(w_{3}f(f(w_{1}w_{2}x))) \\ = f(f(f(w_{1}w_{2}w_{3}x))) \\ = f(f(f(Wx)))$$

여기서 $W = w_{1}w_{2}w_{3}$이다.

<img src="./image/function2.png" alt="function" />

(위 그림과 같이 $w_{i}$가 서로 자리를 바꿀 수 있는 것은 $w_{i}$가 스칼라이기 때문이다. 편의를 위해 순서대로 나열했다. (즉, $w_{1}w_{2} = w_{2}w_{1}$))

이것의 의미는 가중치의 업데이트가 $w_{1}, w_{2}, w_{3}$ 셋 전부에서 일어날 필요가 없다는 것이다.

간단하게 예를 들어 보겠다.

$w_{1}, w_{2}, w_{3}$의 가중치를 모두 $1$로 초기화하고 모델을 훈련시켰을 때, 최종적으로 훈련된 모델의 가중치들이 $w^{'}_{1}, w^{'}_{2}, w^{'}_{3}$라고 하겠다. 이것을 식으로 하면 다음과 같다.

$$y = f(w^{'}_{3}f(w^{'}_{2}f(w^{'}_{1}x)))$$

함수 $f$가 선형인 것을 이용해 식을 다음과 같이 바꾸어 보겠다.

$$y = f(f(f(Wx)))$$

여기서 $W = w^{'}_{1} w^{'}_{2} w^{'}_{3}$이다. 이 식은, 사실상, $w_{1}, w_{2}, w_{3}$의 가중치를 모두 $1$로 초기화하고 모델을 훈련시켰을 때, $w_{2}$와 $w_{3}$ 업데이트 되지 않게 고정시키고 $w_{1}$만 업데이트한 것과 같다.

즉, $w_{2}$나 $w_{3}$의 가중치가 어떻게 변하는지와 상관없이 $w_{1}$만 잘 업데이트되면 결과는 같다는 것이다.

그럼 나가아서 $f(f(f(Wx)))$를 $f(Wx)$로 표현할 수도 있을까?

이렇게 하기 위해선 그냥 $f$함수 3개를 하나의 합성함수로 만들어 주면 된다. 그런데 선형 함수들의 합성함수도 선형일까?

<img src="./image/function3.png" alt="function" />

그렇다. 선형이다. 그럼 어떻게 해서 선형이 되는지 증명해 보고 넘어가도록 하자.

(하나하나 이해하실 필요 없고 이러한 방식으로 증명되었다는 것만 알고 넘어가도 괜찮다.)

#### [선형 변환의 합성함수에 관한 정리]

$V, W$ 그리고 $Z$가 실수 공간상의 벡터 공간이고, 함수 $ \tau  : V \rightarrow W$와 함수 $u : W \rightarrow Z$가 선형이라고 하면, __합성함수__ $ u \tau : V \rightarrow Z$__도 선형이다.__

증명)<br>
$x, y \in V$이고 $\alpha \in \mathbb{R}$이라고 하겠다. 그럼,

$$ u \tau (ax+y) = u(\tau(ax + y)) \\ = u(a\tau(x)+\tau(y)) \\ = au(\tau(x) + u(\tau(y))) \\ = au\tau(x) + u\tau(y)$$

이므로, 선형의 성질에 의해 $u\tau$도 선형이다.

우리는 이제 선형함수의 합성함수 또한 선형이라는 것을 알았다. 이 정리에 의해 우리는 이제 $f(f(f(Wx)))$를 $f(Wx)$로 표현할 수 있다.

이것의 의미는 무엇일까?

바로 3개의 노드를 1개로 줄여서 표현을 해도 결과가 달라지지 않는다는 것이다. 

<img src="./image/function4.png" alt="function" />
